## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

# pv.start_xvfb()


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [10]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0, # Should be between 1.0 and 3.0
    'voxel_size': 0.01, 
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
    'align_floor': False,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    overwrite=True
)


Available runs:
[0] 2025-07-11_171420
Initializing mesher Open3DTSDFFusion


[18:11:01] Auto image downscale factor of 2                                                 ]8;id=584365;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=880447;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[18:11:09] use color only optimization with sigmoid activation                                         ]8;id=661626;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=953309;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

...

Skipping hole Id_EdgeTag(27) of perimeter 45.35377921021609


Filling holes (822): 100%|██████████| 822/822 [00:06<00:00, 131.55it/s]


Mapping normals to mesh


Finished computing mesh: 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/mesh/Open3dTSDFfusion.ply

Mapping features to mesh


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [18]:
splatter.plot_mesh(attribute="RGB")

Number of points: 102294
Number of cells: 200193
Bounds: BoundsTuple(x_min=-1.1563397645950317, x_max=0.9401394128799438, y_min=-0.3644011914730072, y_max=1.4337577819824219, z_min=-0.9725033044815063, z_max=-0.26595816016197205)


Widget(value='<iframe src="http://localhost:41857/index.html?ui=P_0x7ba93c0894e0_8&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [4]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[18:08:28] Auto image downscale factor of 2                                                 ]8;id=439016;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=998197;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[18:08:54] use color only optimization with sigmoid activation                                         ]8;id=137478;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=514754;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Plot similarity maps

In [6]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 102294
Number of cells: 200193
Bounds: BoundsTuple(x_min=-1.1640623807907104, x_max=0.937711775302887, y_min=-0.37410810589790344, y_max=1.4239957332611084, z_min=-0.9620676636695862, z_max=-0.26905032992362976)


Widget(value='<iframe src="http://localhost:36143/index.html?ui=P_0x7ba0dc1dd510_1&reconnect=auto" class="pyvi…

### Try to do it manually

In [9]:
from nerfstudio.utils.eval_utils import eval_setup

_, pipeline, _,  _ = eval_setup(Path(splatter.config['model_config_path']))

[18:10:23] Auto image downscale factor of 2                                                 ]8;id=908979;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=398408;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[18:10:47] use color only optimization with sigmoid activation                                         ]8;id=126828;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=39813;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

In [13]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh(splatter.config['mesh_info']['mesh'])

means = pipeline.model.means.detach().cpu().numpy()

features = pipeline.model.distill_features.detach().cpu().numpy()

In [ ]:
mapped_features = features2vertex(mesh.vertices, means, features)

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
import torch

def features2vertex(mesh_vertices, points, features, k=5, sdf_trunc=0.03):
    """
    Map point cloud features to mesh vertices using KNN over a KDTree.

    Args:
        mesh: final cleaned mesh after culling (mesh_0)
        points: (N, 3) array of input point cloud
        features: (N, D) array of per-point features
        k: number of nearest neighbors to use for weighting
        sdf_trunc: truncation distance for SDF
    
    Returns:
        features_kNN: (M, D) array of per-vertex features
    """

    vertices = np.asarray(mesh_vertices)

    # Build tree
    tree = cKDTree(vertices)

    # Query nearest vertex for each point
    distances, indices = tree.query(points, k=k)  # shape: (N,)

    # Mask points where nearest vertex is within truncation distance
    # Use distance to closest vertex (distance[:, 0]) for truncation mask
    valid_mask = distances[:, 0] <= sdf_trunc

    if not np.any(valid_mask):
        # No points within truncation distance, return zeros
        return np.zeros((len(vertices), features.shape[1]))

    # Filter distances, indices, and features by valid points
    distances = distances[valid_mask]
    indices = indices[valid_mask]
    features = features[valid_mask]

    # Weighting with Gaussian kernel
    sigma = np.mean(distances)  # or set manually
    weights = np.exp(- (distances**2) / (2 * sigma**2))
    weights /= weights.sum(axis=1, keepdims=True)  # normalize

    # Aggregate features per vertex
    features_kNN = np.zeros((len(vertices), features.shape[1]))

    # Use a counts array to normalize contributions per vertex later
    vertex_weight_sum = np.zeros((len(vertices), 1))

    # Accumulate weighted features
    for i in trange(k):
        vertex_indices = indices[:, i]
        weighted_feats = features * weights[:, i:i+1]

        # Accumulate weighted features
        np.add.at(features_kNN, vertex_indices, weighted_feats)

        # Accumulate weights for normalization
        np.add.at(vertex_weight_sum, vertex_indices, weights[:, i:i+1])

    # Normalize aggregated features by summed weights (avoid div by zero)
    nonzero_mask = vertex_weight_sum.squeeze() > 0
    features_kNN[nonzero_mask] /= vertex_weight_sum[nonzero_mask]

    return torch.tensor(features_kNN)
